# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_data = pd.read_csv("output_data/cities.csv", encoding="utf-8")
weather_data.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,albany,95,US,1594941672,58,75.00,42.60,-73.97,1.01
1,1,mataura,16,NZ,1594941672,83,39.00,-46.19,168.86,3.00
2,2,praia da vitoria,75,PT,1594941672,82,68.00,38.73,-27.07,10.80
3,3,edea,75,CM,1594941672,100,75.20,3.80,10.13,3.36
4,4,airai,0,TL,1594941672,84,59.23,-8.93,125.41,2.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
humidity = weather_data["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_data[["Lat", "Lng"]]

In [7]:
# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

NameError: name 'gmaps' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
narrowed_city_df = weather_data.loc[(weather_data["Temp"] > 70) & (weather_data["Temp"] < 80) & (weather_data["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,156,tigzirt,0,DZ,1594941690,76,72.27,36.89,4.12,12.12
1,189,sao joao do piaui,0,BR,1594941693,53,78.67,-8.36,-42.25,8.21
2,204,embonas,0,GR,1594941695,73,77.00,36.23,27.86,11.41
3,214,guerrero negro,0,MX,1594941696,51,77.85,27.98,-114.06,18.30
4,251,eresos,0,GR,1594941484,49,73.40,39.17,25.93,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

NameError: name 'g_key' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
